In [1]:
! pip install joblib

In [1]:
from dask_kubernetes import KubeCluster

cluster = KubeCluster.from_yaml('worker-spec.yml')
cluster.adapt(minimum=1, maximum=10)  # or dynamically scale based on current workload

distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:   tcp://10.42.13.90:33505
distributed.scheduler - INFO -   dashboard at:                     :8787
distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=1 maximum=10


In [19]:
from aflow import search,K
from joblib import Memory

def get_initial_aflow_results(nspecies=2,enthalpy_formation_atom=-0.1):
    result = search().select(K.nspecies==nspecies).filter(K.enthalpy_formation_atom<enthalpy_formation_atom)
    return result

location = './cachedir'
memory = Memory(location,verbose=1)
get_initial_aflow_results = memory.cache(get_initial_aflow_results)

In [41]:
import pickle
with open('all_results.pkl','rb') as filehandle:
    all_aflow_binaries = pickle.load(filehandle)
with open('bimetallics.pkl','rb') as filehandle:
    bimetallics = pickle.load(filehandle)

In [42]:
import dask.bag as db 
from surface_nuclearity_calculator import bulk_nuclearity, select_bimetallic

actives = ['Pd', 'Pt', 'Rh', 'Ru', 'Ag', 'Ir','Pd\n', 'Pt\n', 'Rh\n', 'Ru\n', 'Ag\n', 'Ir\n']
hosts = ['Zn', 'Cd', 'Ga', 'Al', 'In','Zn\n', 'Cd\n', 'Ga\n', 'Al\n', 'In\n']

print("getting intital results")
#all_aflow_binaries = get_initial_aflow_results(enthalpy_formation_atom=-3.1)
#all_aflow_binaries_bag = db.from_sequence(all_aflow_binaries)
print("applying filter")
#active_inactive_bag = all_aflow_binaries_bag.filter(lambda r: select_bimetallic(r,actives,hosts)>0).map(lambda b: bulk_nuclearity(b,actives)).compute()#.take(2)#joblib later
print("nuclearity calculation")
#outputs=active_inactive_bag.map(lambda b: bulk_nuclearity(b,actives)).compute()
outputs=db.from_sequence(bimetallics).map(lambda b: bulk_nuclearity(b,actives)).compute()

getting intital results
applying filter
nuclearity calculation


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)


KeyboardInterrupt: 

In [40]:
active_inactive_bag

[]

In [29]:
out=db.from_sequence([1,2,3,4]).filter(lambda q: q).map(square).compute()

In [30]:
out

[1, 4, 9, 16]

In [18]:
type(bimetallics[0])

aflow.entries.Entry

In [26]:
def square(x): 
    return x**2

In [6]:
all_a

[]